In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Dog&Cat  SVM

---

### Team members:
* Ziyi Wang   ID 18042783
* Youzhi Lei  ID 19039281

## Content
* Using SVM to classify the dog&cat

---


#### Information about datasets
#### Labeled dogs and cats from Microsoft
* Dogs and Cats image https://www.microsoft.com/en-us/download/details.aspx?id=54765

In [2]:
#!pip install opencv-python

In [3]:
#!pip install tensorflow

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import patsy
import os
from PIL import Image 
import cv2
%matplotlib inline

In [2]:
# import the necessary packages
import sklearn
print(sklearn.__version__)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import cv2
import os

0.20.2


In [3]:
def image_to_feature_vector(image, size=(50, 50)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    # handle normalizing the histogram if we are using OpenCV 2.4.X
    # if imutils.is_cv2():
    #    hist = cv2.normalize(hist)
    #print("line2")
    # otherwise, perform "in place" normalization in OpenCV 3
    #else:
    cv2.normalize(hist, hist)
    
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [4]:
from pylab import rcParams

sns.set(style="ticks")
#sns.set_style("whitegrid")
rcParams['figure.dpi'] = 150
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['patch.edgecolor'] = 'white'
rcParams['font.family'] = 'StixGeneral'
rcParams['figure.figsize'] = 10,8
rcParams['font.size'] = 14
rcParams['axes.labelsize'] = 'large'
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14

pd.options.display.max_columns = 50

In [5]:
import os
cwd = os.getcwd()
print(cwd)
#Please put the data to current location, Thanks 

/home/dp1/Documents/fred/codes/machinelearning2019/dogcat


In [6]:
#load the dataset 
#test one image and show 
DATADIR = "../kagglecatsanddogs/PetImages"
DATADIR = os.path.join(cwd, DATADIR)
print(DATADIR)
CATEGORIES = ["Dog","Cat"]


/home/dp1/Documents/fred/codes/machinelearning2019/dogcat/../kagglecatsanddogs/PetImages


In [7]:
rawImages = []
features = []
labels = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        #conver the dog and cat to numerical value 0/1
        label = CATEGORIES.index(category)
        
        for img in os.listdir(path):
            try:
                #read  
                imgpath = os.path.join(path,img)
                
                img_array = cv2.imread(imgpath)

                pixels = image_to_feature_vector(img_array)
                
                hist = extract_color_histogram(img_array)
                
                
            except Exception as e:
                print(imgpath)
                os.remove(imgpath) 
                continue
                
            rawImages.append(pixels)
            features.append(hist)
            labels.append(label)

            
create_training_data()

In [8]:
print(len(rawImages))
print(len(features))
print(len(labels))


24946
24946
24946


In [9]:
# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
print("[INFO] pixels matrix: {:.2f}MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(features.nbytes / (1024 * 1000.0)))


[INFO] pixels matrix: 182.71MB
[INFO] features matrix: 49.89MB


In [15]:
from sklearn import preprocessing
#features_scaled = preprocessing.scale(features)
scaler = StandardScaler()
features_scaled  = scaler.fit(features)

NameError: name 'StandardScaler' is not defined

In [11]:
# partition the data into training and testing splits, using 85%
# of the data for training and the remaining 15% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(rawImages, labels, test_size=0.15, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(features, labels, test_size=0.15, random_state=42)
#(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(features_scaled, labels, test_size=0.15, random_state=42)

In [ ]:
import pickle
dataset = {
    'trainRI': trainRI,
    'testRI' : testRI,
    'trainRL': trainRL,
    'testRL' : testRL
}
filename = 'rawImagesdataset.pickle'

outfile = open(filename,'wb+')
print(outfile)
pickle.dump(dataset,outfile)
outfile.close()


In [ ]:
import pickle

dataset = {
    'trainFeat': trainFeat,
    'testFeat' : testFeat,
    'trainLabels': trainLabels,
    'testLabels' : testLabels
}
filename = 'features_scaleddataset.pickle'

outfile = open(filename,'wb+')
print(outfile)
pickle.dump(dataset,outfile)
outfile.close()


In [12]:
print(trainRI.shape)
print(testRI.shape)
print(trainRL.shape)
print(testRL.shape)

print(trainFeat.shape)
print(testFeat.shape)
print(trainLabels.shape)
print(testLabels.shape)

print(type(trainRI))
print(type(testRI))
print(type(trainRL))
print(type(testRL))

print(type(trainFeat))
print(type(testFeat))
print(type(trainLabels))
print(type(testLabels))

print(testFeat)
print(testLabels)

(21204, 7500)
(3742, 7500)
(21204,)
(3742,)
(21204, 512)
(3742, 512)
(21204,)
(3742,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[7.7037728e-01 8.6041335e-03 1.5518170e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.9476840e-02 1.5552505e-02 1.6915949e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.2886506e-02 1.0173556e-02 2.5128685e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [6.3553471e-03 3.2998919e-03 2.1999279e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [6.5048292e-02 1.4669396e-02 2.7075514e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.9178828e-03 2.0058766e-02 1.1740961e-01 ... 1.5489395e-04
  0.0000000e+00 0.0000000e+00]]
[1 0 0 ... 1 1 0]


In [20]:
testFeat = np.array(testFeat)
testLabels = np.array(testLabels)

In [21]:
print(type(testFeat))
print(type(testLabels))

print(testFeat)
print(testLabels)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[9.0343070e-01 2.4944368e-01 6.7670859e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 4.0895315e-03 7.9453751e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [8.2122593e-04 5.1619918e-03 1.8418925e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 4.2775902e-04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.1264091e-02 1.3338332e-01 5.0276309e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.7554853e-02 4.9082082e-02 2.2388319e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
[1 0 0 ... 0 1 0]


In [22]:
# k-NN
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1)
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
#print("[INFO] k-NN classifier: k=%d" % args["neighbors"])
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))




[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 55.05%


In [34]:
#neural network
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, alpha=1e-4,
                      solver='sgd', tol=1e-4, random_state=1,
                      learning_rate_init=.1)
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
print("[INFO] neural network raw pixel accuracy: {:.2f}%".format(acc * 100))

In [ ]:
#neural network
print("\n")
print("[INFO] evaluating histogram accuracy...")
model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, alpha=1e-4,
                      solver='sgd', tol=1e-4, random_state=1,
                      learning_rate_init=.1)
model.fit(trainFeat, trainLabels)
acc = model.score(testFeat, testLabels)
print("[INFO] neural network histogram accuracy: {:.2f}%".format(acc * 100))


In [ ]:
import pickle

with open('rawImagesdataset.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    rawImagesdataset = pickle.load(f)
    f.close()
    
(trainRI, testRI, trainRL, testRL) =   rawImagesdataset  

In [ ]:
#SVC
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = SVC(max_iter=1000,class_weight='balanced')
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
print("[INFO] SVM-SVC raw pixel accuracy: {:.2f}%".format(acc * 100))




[INFO] evaluating raw pixel accuracy...


/usr/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
import pickle

with open('features_scaleddataset.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    features_scaleddataset = pickle.load(f)
    f.close()
    
(trainRI, testRI, trainRL, testRL) =   features_scaleddataset  

In [13]:
#SVC
print("\n")
print("[INFO] evaluating histogram accuracy...")
model = SVC(max_iter=1000,class_weight='balanced')
model.fit(trainFeat, trainLabels)
acc = model.score(testFeat, testLabels)
print("[INFO] SVM-SVC histogram accuracy: {:.8f}%".format(acc * 100))



[INFO] evaluating histogram accuracy...


/usr/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[INFO] SVM-SVC histogram accuracy: 51.01549973%
